# 粒子群优化

----

## 任务目的

熟悉和掌握粒子群优化算法的核心原理和概念，包括粒子适应度、粒子位置、粒子速度等，掌握粒子位置和速度迭代更新方法。

## 任务内容

编写粒子群优化算法，实现多维数据点的求解。本次实验将撰写相关代码求解粒子的个体最佳位置、群体最佳位置，迭代更新粒子的位置和速度，最终求得在多维坐标下目标函数为某一值时的坐标位置。最后验证所设计粒子群优化算法的正确性。  
<a href='#目标函数与参数初始化'>目标函数与参数初始化</a><br/>
<a href='#粒子迭代'>粒子迭代</a><br/>

## 任务原理

粒子群算法（Particle Swarm Optimization, PSO)）是群智能的一种，是基于对鸟群觅食行为的研究和模拟而来的。假设在鸟群觅食范围，只在一个地方有食物，所有鸟儿看不到食物（不知道食物的具体位置），但是能闻到食物的味道（能知道食物距离自己位置）。最好的策略就是结合自己的经验在距离鸟群中距离食物最近的区域搜索。  
鸟被抽象为没有质量和体积的微粒(点)，并延伸到 维空间，粒子 在 维空间的位置表示为矢量 ，飞行速度表示为矢量 。每个粒子都有一个由目标函数决定的适应值，并且知道自己到目前为止发现的最好位置(pbest)和现在的位置，这个可以看作是粒子自己的飞行经验。除此之外，每个粒子还知道到目前为止整个群体中所有粒子发现的最好位置(gbest)(gbest是pbest中的最好值)，这个可以看作是粒子同伴的经验。粒子就是通过自己的经验和同伴中最好的经验来决定下一步的运动。  
利用粒子群算法解决实际问题本质上就是利用粒子群算法求解函数的最值。因此需要事先把实际问题抽象为一个数学函数，称之为适应度函数。在粒子群算法中，每只鸟都可以看成是问题的一个解，这里我们通常把鸟称之为粒子，每个粒子都拥有：  
位置：可以理解为函数自变量的值；  
经验：也即是自身经历过的距离食物最近的位置；   
速度：可以理解为自变量的变化值；  
适应度：距离食物的位置，也就是函数值。  
粒子群算法的过程大致分为：初始化、计算适应度、更新状态、判断结束条件。标准粒子群PSO初始化为一群随机粒子(随机解)，然后通过迭代找到最优解。在每一次的迭代中，粒子通过跟踪两个“极值”(pbest，gbest)来更新自己。在找到这两个最优值后，粒子通过下面的公式来更新自己的速度和位置。
<img src="../../image/MachineLearningAlgorithm/1-3.png" width="450" length="300"> 
在以上公式中，$i=1,2,...,N$，$N$是此群中粒子的总数，$v_i$是粒子的速度，rand()生成随机数，$x_i$为粒子的当前位置，$c_1$、$c_2$是学习因子，通常$c_1=c_2=2$。 
第一个公式的第一部分称为记忆项，表示上次速度大小和方向的影响；公式的第二部分称为自身认知项，是从当前点指向粒子自身最好点的一个矢量，表示粒子的动作来源于自己经验的部分；公式的第三部分称为群体认知项，是一个从当前点指向种群最好点的矢量，反映了粒子间的协同合作和知识共享。粒子就是通过自己的经验和同伴中最好的经验来决定下一步的运动。  
标准粒子群PSO算法流程：  
1、初始化一群微粒(群体规模为N)，包括随机位置和速度；  
2、评价每个微粒的适应度；  
3、对每个微粒，将其适应值与其经过的最好位置pbest作比较，如果较好，则将其作为当前的最好位置pbest；  
4、对每个微粒，将其适应值与其经过的最好位置gbest作比较，如果较好，则将其作为当前的最好位置gbest；  
5、根据以上公式调整微粒速度和位置；  
6、未达到结束条件则转第2步。  

## 任务步骤

1、定义粒子群类PSO，构造函数定义了粒子群参数，包括了粒子数量、粒子维度、粒子速度、位置等，pbest为粒子个体经历的最佳位置，gbest为全局最佳位置。

```python
import numpy as np  
import random   
import matplotlib.pyplot as plt  
import math
import pdb
class PSO:
    def __init__(self,pN,dim,max_iter): 
# PSO参数设置
        self.w = 1    
        self.c1 = 2     
        self.c2 = 2     
        self.r1= 0.6  
        self.r2=0.3  
        self.pN = pN                #粒子数量  
        self.dim = dim              #搜索维度  
        self.max_iter = max_iter    #迭代次数  
        self.X = np.zeros((self.pN,self.dim)) #所有粒子的位置和速度  
        self.V = np.zeros((self.pN,self.dim)) 
#个体经历的最佳位置和全局最佳位置
        self.pbest = np.zeros((self.pN,self.dim))+10  
        self.gbest = np.zeros((1,self.dim))+10  
        self.p_fit = np.zeros(self.pN)+10  #每个个体的历史最佳适应值  
        self.fit = 8             #全局最佳适应值  
        self.fit_value = 10
```

### 目标函数与参数初始化

2、定义目标函数。

```python
# 目标函数Sphere函数
    def function(self,x):  
        sum_value = 0  
        length = len(x)  
        x2 = 0
        for i in range(length):
            x2 = 10*math.sin(5*x[i]) + 7 * math.cos(4 * x[i])
            sum_value += x2 
        return sum_value

```

3、初始化种群，初始化粒子的位置和速度，初始化粒子个体最佳位置、全局最佳位置。

```python
#初始化种群  
    def init_Population(self):  
        for i in range(self.pN):  
            for j in range(self.dim):  
                self.X[i][j] = random.uniform(0,1)  
                self.V[i][j] = random.uniform(0,1)  
            self.pbest[i] = self.X[i]  
            tmp = self.function(self.X[i])  
            self.p_fit[i] = abs(tmp  - self .fit_value)
            if(abs(tmp-self.fit_value) < self.fit):  
                self.fit = abs(tmp  - self.fit_value)
                self.gbest = self.X[i]  
```

### 粒子迭代

4、粒子迭代更新如下所示，以下迭代实现了上文中定义的目标函数取值为10时的粒子位置，迭代求解当前粒子经历最佳位置并从所有最佳位置中求解全局最佳位置。所有粒子遍历后，按照实验原理中公式更新每个粒子的位置和速度，并开始下一轮的迭代。

```python
#更新粒子位置  
    def iterator(self):  
        fitness = []  
        for t in range(self.max_iter):  
            for i in range(self.pN):         #更新gbest\pbest
               
               temp = self.function(self.X[i])
               
               if( abs(temp-self.fit_value) < self.p_fit[i]):      #更新个体最优
                   
                   self.p_fit[i] = abs(temp - self.fit_value)
                   self.pbest[i] = self.X[i].copy()
                   if(self.p_fit[i] < self.fit):  #更新全局最优
                       self.gbest = self.X[i].copy()
                       self.fit = self.p_fit[i].copy()  
                       
            for i in range(self.pN):  
                self.V[i] = self.w*self.V[i] + self.c1*self.r1*(self.pbest[i] - self.X[i])\
                            + self.c2*self.r2*(self.gbest - self.X[i])  
                self.X[i] = self.X[i] + self.V[i]
            #pdb.set_trace()
            fitness.append(self.fit)
        print('best=%',self.gbest)
        print('iter=%',t)
        return self.gbest,fitness  
```

5、执行步骤如下所示，粒子类定义、初始话和迭代求解。最后画出粒子优化算法的数字求解过程。 

```python
#程序执行  
my_pso = PSO(pN=30,dim=5,max_iter=200)  
my_pso.init_Population()  
gbest,fitness = my_pso.iterator()  
#画图  
plt.figure(1)  
plt.title("ParticleSwarmOptimization")  
plt.xlabel("iterators", size=14)  
plt.ylabel("fitness", size=14)  
leniter = len(fitness)
t = np.array([t for t in range(leniter)])  
fitness = np.array(fitness)  
plt.plot(t,fitness, color='b',linewidth=3)  
plt.show() 
```

6、验证所设计粒子优化算法的正确性，如下所示，将求解得到的粒子最佳位置带入目标函数。

```python
x=gbest
sum_value = 0
for i in range(5):
    x[i] = 10*math.sin(5*x[i]) + 7 * math.cos(4 * x[i])
    sum_value += x[i]  
print("the value is %f" %sum_value)
```

7、得到输出，可观察到，随着迭代次数的增加，误差值越来越小。验证得到的输出值为10.000872，与fit_value的值几乎相等。

8、以上过程在迭代的时候指定可迭代次数，另外一种方式是，迭代次数和目标误差值的组合方式，采用该方式，可减小不别要的计算量同时能满足设计需求。在PSO类中添加error参数，设置为0.008。在迭代函数中添加if判断语句，判断当前误差值是否满足最小误差，此外最大迭代次数可保证函数无限次迭代下去。主要需要修改画图参数，此时fitness长度不确定。最后执行得到，只需要迭代122次可满足误差。   
利用以下的的代码替换上文中相应的代码段，再次运行即可。

```python
class PSO:
    def __init__(self,pN,dim,max_iter): 
# PSO参数设置
        self.w = 1    
        self.c1 = 2     
        self.c2 = 2     
        self.r1= 0.6  
        self.r2=0.3  
        self.pN = pN                #粒子数量  
        self.dim = dim              #搜索维度  
        self.max_iter = max_iter    #迭代次数  
        self.X = np.zeros((self.pN,self.dim)) #所有粒子的位置和速度  
        self.V = np.zeros((self.pN,self.dim)) 
#个体经历的最佳位置和全局最佳位置
        self.pbest = np.zeros((self.pN,self.dim))+10  
        self.gbest = np.zeros((1,self.dim))+10  
        self.p_fit = np.zeros(self.pN)+10  #每个个体的历史最佳适应值  
        self.fit = 8             #全局最佳适应值  
        self.fit_value = 10
        self.error = 0.008
```

```python
#更新粒子位置  
    def iterator(self):  
        fitness = []  
        for t in range(self.max_iter):  
            for i in range(self.pN):         #更新gbest\pbest
               
               temp = self.function(self.X[i])
               
               if( abs(temp-self.fit_value) < self.p_fit[i]):      #更新个体最优
                   
                   self.p_fit[i] = abs(temp - self.fit_value)
                   self.pbest[i] = self.X[i].copy()
                   if(self.p_fit[i] < self.fit):  #更新全局最优
                       self.gbest = self.X[i].copy()
                       self.fit = self.p_fit[i].copy()  
                       
            for i in range(self.pN):  
                self.V[i] = self.w*self.V[i] + self.c1*self.r1*(self.pbest[i] - self.X[i])\
                            + self.c2*self.r2*(self.gbest - self.X[i])  
                self.X[i] = self.X[i] + self.V[i]
            #pdb.set_trace()
            fitness.append(self.fit)  
            if(self.fit <= self.error):
                break
        print('best=%',self.gbest)
        print('iter=%',t)
        return self.gbest,fitness  
```